In [ ]:
import os
from dotenv import load_dotenv
from huggingface_hub import HfApi, create_repo
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
from dataclasses import dataclass
from torch.amp import autocast, GradScaler
import torch.utils.checkpoint

try:
    import datasets
    import transformers
except ImportError:
    print("Installing dependencies...")
    import subprocess
    subprocess.check_call(["pip", "install", "-q", "datasets", "transformers", "accelerate"])
    from datasets import load_dataset, concatenate_datasets
    from transformers import AutoTokenizer
else:
    from datasets import load_dataset, concatenate_datasets
    from transformers import AutoTokenizer


#load_dotenv()


if not HF_TOKEN or not REPO_NAME:
    raise ValueError("Error: HF_TOKEN or REPO_NAME not found in .env file.")


api = HfApi(token=HF_TOKEN)
try:
    create_repo(repo_id=REPO_NAME, repo_type="model", token=HF_TOKEN, exist_ok=True)
    print(f"Connected to Hugging Face Repo: {REPO_NAME}")
except Exception as e:
    print(f" Repo Connection Failed: {e}")


@dataclass
class GemmaZeroConfig:
    vocab_size: int = 50257      # Must be 50257 for GPT2
    hidden_size: int = 1024      
    intermediate_size: int = 4096 
    num_hidden_layers: int = 24  
    num_attention_heads: int = 16 
    num_key_value_heads: int = 4 
    head_dim: int = 64
    max_position_embeddings: int = 2048 
    rms_norm_eps: float = 1e-6
    rope_theta: float = 10000.0
    attn_logit_softcapping: float = 50.0
    final_logit_softcapping: float = 30.0


Connected to Hugging Face Repo: FusionCorp/gemma-zero


In [2]:
class GemmaRMSNorm(nn.Module):
    def __init__(self, dim: int, eps: float = 1e-6):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.ones(dim))
    def forward(self, x):
        x_float = x.float()
        variance = x_float.pow(2).mean(-1, keepdim=True)
        x_float = x_float * torch.rsqrt(variance + self.eps)
        return (x_float * self.weight.float()).type_as(x) + 1.0

class GemmaRotaryEmbedding(nn.Module):
    def __init__(self, dim, max_position_embeddings=2048, base=10000, device=None):
        super().__init__()
        inv_freq = 1.0 / (base ** (torch.arange(0, dim, 2).float().to(device) / dim))
        self.register_buffer("inv_freq", inv_freq, persistent=False)
    def forward(self, x, seq_len=None):
        t = torch.arange(seq_len, device=x.device, dtype=self.inv_freq.dtype)
        freqs = torch.einsum("i,j->ij", t, self.inv_freq)
        emb = torch.cat((freqs, freqs), dim=-1)
        return emb.cos(), emb.sin()

def apply_rotary_pos_emb(q, k, cos, sin):
    def rotate_half(x): return torch.cat((-x[..., x.shape[-1] // 2:], x[..., :x.shape[-1] // 2]), dim=-1)
    return (q * cos) + (rotate_half(q) * sin), (k * cos) + (rotate_half(k) * sin)

class GemmaAttention(nn.Module):
    def __init__(self, config: GemmaZeroConfig):
        super().__init__()
        self.config = config
        self.num_heads = config.num_attention_heads
        self.head_dim = config.head_dim
        self.num_key_value_heads = config.num_key_value_heads
        self.num_key_value_groups = self.num_heads // self.num_key_value_heads
        
        self.q_proj = nn.Linear(config.hidden_size, self.num_heads * self.head_dim, bias=False)
        self.k_proj = nn.Linear(config.hidden_size, self.num_key_value_heads * self.head_dim, bias=False)
        self.v_proj = nn.Linear(config.hidden_size, self.num_key_value_heads * self.head_dim, bias=False)
        self.o_proj = nn.Linear(self.num_heads * self.head_dim, config.hidden_size, bias=False)
        self.rotary_emb = GemmaRotaryEmbedding(self.head_dim, config.max_position_embeddings, config.rope_theta)

    def forward(self, hidden_states, attention_mask=None):
        bsz, q_len, _ = hidden_states.size()
        q = self.q_proj(hidden_states).view(bsz, q_len, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.k_proj(hidden_states).view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)
        v = self.v_proj(hidden_states).view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)
        
        cos, sin = self.rotary_emb(v, seq_len=q_len)
        q, k = apply_rotary_pos_emb(q, k, cos, sin)
        k, v = k.repeat_interleave(self.num_key_value_groups, dim=1), v.repeat_interleave(self.num_key_value_groups, dim=1)
        
        attn_weights = torch.matmul(q, k.transpose(2, 3)) / math.sqrt(self.head_dim)
        if self.config.attn_logit_softcapping:
            attn_weights = torch.tanh(attn_weights / self.config.attn_logit_softcapping) * self.config.attn_logit_softcapping
        if attention_mask is not None: attn_weights = attn_weights + attention_mask
        
        attn_output = torch.matmul(F.softmax(attn_weights, dim=-1, dtype=torch.float32).to(q.dtype), v)
        return self.o_proj(attn_output.transpose(1, 2).contiguous().view(bsz, q_len, -1))

class GemmaBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.input_layernorm = GemmaRMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self.self_attn = GemmaAttention(config)
        self.post_attention_layernorm = GemmaRMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self.mlp = nn.Sequential(
            nn.Linear(config.hidden_size, config.intermediate_size, bias=False), 
            nn.Linear(config.hidden_size, config.intermediate_size, bias=False), 
            nn.Linear(config.intermediate_size, config.hidden_size, bias=False) 
        )
        self.mlp_gate = self.mlp[0]; self.mlp_up = self.mlp[1]; self.mlp_down = self.mlp[2]

    def forward(self, x, mask=None):
        r = x; x = self.input_layernorm(x); x = self.self_attn(x, attention_mask=mask); x = r + x
        r = x; x = self.post_attention_layernorm(x)
        gate, val = self.mlp_gate(x), self.mlp_up(x)
        x = self.mlp_down(F.gelu(gate) * val)
        return r + x

class GemmaZeroModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.embed_tokens = nn.Embedding(config.vocab_size, config.hidden_size)
        self.layers = nn.ModuleList([GemmaBlock(config) for _ in range(config.num_hidden_layers)])
        self.norm = GemmaRMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self.embed_scale = math.sqrt(config.hidden_size)
        self.gradient_checkpointing = False

    def gradient_checkpointing_enable(self): self.gradient_checkpointing = True

    def forward(self, input_ids):
        x = self.embed_tokens(input_ids) * self.embed_scale
        for layer in self.layers:
            if self.gradient_checkpointing and self.training:
                # This line saves ~10GB of VRAM by re-calculating 
                # activations during the backward pass.
                x = torch.utils.checkpoint.checkpoint(
                    layer, 
                    x, 
                    None,
                    use_reentrant=False
                )
            else:
                x = layer(x)
        x = self.norm(x)
        logits = torch.matmul(x, self.embed_tokens.weight.t())
        if self.config.final_logit_softcapping:
             logits = torch.tanh(logits / self.config.final_logit_softcapping) * self.config.final_logit_softcapping
             
        return logits

In [3]:
def get_tinystories_loader(batch_size=4, seq_len=2048):
    print("Loading TinyStories...")
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token
    
    # TinyStories is massive, so we stream it
    dataset = load_dataset("roneneldan/TinyStories", split="train", streaming=True)
    
    def data_generator():
        for item in dataset:
            # Filter out very short stories
            if len(item['text']) < 50: continue
            
            # Pack/Truncate
            tokens = tokenizer.encode(item['text'], max_length=seq_len, truncation=True, padding="max_length")
            yield torch.tensor(tokens)
            
    gen = data_generator()
    def get_batch():
        batch = [next(gen) for _ in range(batch_size)]
        return torch.stack(batch).cuda()
    return get_batch


In [ ]:

!pip install -q bitsandbytes 

from transformers import get_linear_schedule_with_warmup # Add this import

def train():
    device = "cuda"
    torch.cuda.empty_cache()
    
    config = GemmaZeroConfig() 
    model = GemmaZeroModel(config).to(device)
    model.gradient_checkpointing_enable() 

    
    optimizer = bnb.optim.AdamW8bit(model.parameters(), lr=3e-4, weight_decay=0.01)
    scaler = GradScaler(device='cuda')

    BATCH_SIZE = 2      
    ACCUM_STEPS = 16    
    SEQ_LEN = 2048
    TOTAL_STEPS = 5000
    
     ADD SCHEDULER (Warmup for 100 steps)
    
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=100, 
        num_training_steps=TOTAL_STEPS
    )
    
    get_batch = get_tinystories_loader(BATCH_SIZE, SEQ_LEN)
    pad_token_id = 50256

    model.train()
    optimizer.zero_grad(set_to_none=True)
    
    print(f"🚀 Training 400M Model | Stability Mode: ON")

    for step in range(TOTAL_STEPS):
        inputs = get_batch()
        labels = inputs.clone()
        
        # SOTA Masking
        labels[labels == pad_token_id] = -100

        with autocast(device_type='cuda', dtype=torch.float16): 
            logits = model(inputs)
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            
            loss = F.cross_entropy(
                shift_logits.view(-1, config.vocab_size), 
                shift_labels.view(-1),
                ignore_index=-100 
            )
            loss = loss / ACCUM_STEPS
        
        scaler.scale(loss).backward()
        
        if (step + 1) % ACCUM_STEPS == 0:
            # 3. ADD GRADIENT CLIPPING (The Stabilizer)
            # This prevents the loss from exploding to 12.0+
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            scaler.step(optimizer)
            scaler.update()
            
            # Step the scheduler
            scheduler.step()
            
            optimizer.zero_grad(set_to_none=True)
            
            # Monitoring
            if (step + 1) % (ACCUM_STEPS * 2) == 0:
                mem = torch.cuda.max_memory_allocated() / 1e9
                print(f"Step {step+1} | Loss: {loss.item() * ACCUM_STEPS:.4f} | Peak VRAM: {mem:.2f} GB")
        
        # Upload Logic (Same as before)
        if (step + 1) % 200 == 0:
            # ... (Your existing upload code here)
            pass 

if __name__ == "__main__":
    train()

Loading TinyStories...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

 Training 400M Model | Brain Density: 24 Layers
Effective Batch: 32 | Optimizer: 8-bit AdamW
Step 32 | Loss: 26.7978 | Peak VRAM: 8.04 GB
Step 64 | Loss: 21.8812 | Peak VRAM: 8.04 GB
Step 96 | Loss: 19.9212 | Peak VRAM: 8.04 GB
Step 128 | Loss: 17.2931 | Peak VRAM: 8.04 GB
Step 160 | Loss: 14.2393 | Peak VRAM: 8.04 GB
Step 192 | Loss: 16.1883 | Peak VRAM: 8.04 GB
☁️ Uploading Checkpoint to FusionCorp/gemma-zero...


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  pytorch_model.bin           :   0%|          |  557kB / 1.67GB            

Upload Success at Step 600
Step 608 | Loss: 12.0771 | Peak VRAM: 8.04 GB


KeyboardInterrupt: 